In [53]:
import tensorflow as tf
import pandas as pd
import tensorflow.keras.layers as layers

In [54]:
dataset_path = tf.keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight', 'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
na_values = "?", comment='\t',sep=" ", skipinitialspace=True)
dataset = raw_dataset.copy()

In [55]:
dataset.head()


,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
0,18.0,8,307.0,130.0,3504.0,12.0,70,1
1,15.0,8,350.0,165.0,3693.0,11.5,70,1
2,18.0,8,318.0,150.0,3436.0,11.0,70,1
3,16.0,8,304.0,150.0,3433.0,12.0,70,1
4,17.0,8,302.0,140.0,3449.0,10.5,70,1


In [56]:
dataset.isna().sum() # Calculate the number of missing values
dataset = dataset.dropna() # Drop missing value records
dataset.isna().sum() # Calculate the number of missing

MPG             0
Cylinders       0
Displacement    0
Horsepower      0
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

In [57]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0

In [58]:
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790.0,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130.0,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295.0,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625.0,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720.0,19.4,82,1.0,0.0,0.0


In [59]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [60]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

In [61]:
train_stats = train_dataset.describe()

train_stats = train_stats.transpose()

In [62]:
def norm(x): # minus mean and divide by std
 return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [63]:
print(normed_train_data.shape,train_labels.shape)
print(normed_test_data.shape, test_labels.shape)

(314, 9) (314,)
(78, 9) (78,)


In [64]:
train_db = tf.data.Dataset.from_tensor_slices((normed_train_data.values, train_labels.values))
train_db = train_db.shuffle(100).batch(32)

In [65]:
class Network(tf.keras.Model):
 # regression network
 def __init__(self):
    super(Network, self).__init__()
 # create 3 fully-connected layers
    self.fc1 = layers.Dense(64, activation='relu')
    self.fc2 = layers.Dense(64, activation='relu')
    self.fc3 = layers.Dense(1)
 def call(self, inputs, training=None, mask=None):
 # pass through the 3 layers sequentially
    x = self.fc1(inputs)
    x = self.fc2(x)
    x = self.fc3(x)
    return x

In [66]:
model = Network()
model.build(input_shape=(4, 9))

In [67]:
model.summary()

Model: "network"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               multiple                  640       
                                                                 
 dense_1 (Dense)             multiple                  4160      
                                                                 
 dense_2 (Dense)             multiple                  65        
                                                                 
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________


In [68]:
optimizer = tf.keras.optimizers.RMSprop(0.001)

In [70]:
for epoch in range(200): # 200 Epoch
 for step, (x,y) in enumerate(train_db): # Loop through training set once
 # Set gradient tape
    with tf.GradientTape() as tape:
        out = model(x) # Get network output
        loss = tf.reduce_mean(tf.keras.losses.MSE(y, out)) 
# Calculate MSE
        mae_loss = tf.reduce_mean(tf.keras.losses.MAE(y, out)) 
# Calculate MAE
    if step % 10 == 0: # Print training loss every 10 steps
        print(epoch, step, float(loss))
 # Calculate and update gradients
 grads = tape.gradient(loss, model.trainable_variables)
 optimizer.apply_gradients(zip(grads, model.trainable_variables))

0 0 665.4410400390625
1 0 632.283447265625
2 0 681.2919921875
3 0 600.387939453125
4 0 628.412109375
5 0 518.6058349609375
6 0 537.4845581054688
7 0 546.89306640625
8 0 530.9337768554688
9 0 515.596435546875
10 0 697.335693359375
11 0 606.0731811523438
12 0 545.90283203125
13 0 452.21246337890625
14 0 647.2406005859375
15 0 515.1156616210938
16 0 551.9862670898438
17 0 572.2100830078125
18 0 476.64654541015625
19 0 504.569580078125
20 0 415.70758056640625
21 0 453.50482177734375
22 0 504.2904968261719
23 0 506.0628662109375
24 0 465.39874267578125
25 0 480.8548278808594
26 0 333.3955078125
27 0 493.22454833984375
28 0 436.05035400390625
29 0 423.82806396484375
30 0 350.58648681640625
31 0 444.6851806640625
32 0 494.9124755859375
33 0 476.0270080566406
34 0 348.00103759765625
35 0 338.85870361328125
36 0 368.003662109375
37 0 393.74493408203125
38 0 301.124755859375
39 0 377.169677734375
40 0 435.079345703125
41 0 253.91513061523438
42 0 324.1486511230469
43 0 363.87054443359375
44 0 34